# Day 7

In [1]:
# lines = [line.rstrip() for line in open('day7_input.txt')]
lines = [line.rstrip() for line in open('day7_sample.txt')]

In [2]:
hands = [line.split(' ')[0] for line in lines]
bids = [int(line.split(' ')[1]) for line in lines]

In [3]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.10f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap

## Part 1

In [4]:
ranked_cards = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
replacements = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [5]:
def determine_type(hand):
    set_hand = set(hand)
    list_set_hand = list(set_hand)
    if len(set_hand)==1:
        return 0
    if len(set_hand)==2:
        if hand.count(list_set_hand[0])==4 or hand.count(list_set_hand[0])==1:
            return 1
        else:
            return 2
    if len(set_hand)==3:
        if hand.count(list_set_hand[0])==3 or hand.count(list_set_hand[1])==3 or hand.count(list_set_hand[2])==3:
            return 3
        else:
            return 4
    if len(set_hand)==4:
        return 5
    if len(set_hand)==5:
        return 6

def replace_hand(hand):
    new_hand = []
    for card in hand:
        new_hand.append(replacements[ranked_cards.index(card)])
    return new_hand

In [6]:
def order_list(some_list):
    hands = [item[0] for item in some_list]
    bids = [item[1] for item in some_list]
    hands_sorted=sorted(hands, key=lambda x: (x[0], x[1], x[2], x[3], x[4]))
    return [bids[hands.index(hand)] for hand in hands_sorted ]

In [7]:
@timing
def total_winnings():
    five_of_a_kinds = []
    four_of_a_kinds = []
    full_houses = []
    three_of_a_kinds = []
    two_pairs = []
    one_pairs = []
    high_cards = []
    
    for i, hand in enumerate(hands):
        hand_type = determine_type(hand)
        replaced_hand = replace_hand(hand)
        if hand_type == 0:
            five_of_a_kinds.append((replaced_hand, bids[i]))
        elif hand_type == 1:
            four_of_a_kinds.append((replaced_hand, bids[i]))
        elif hand_type==2:
            full_houses.append((replaced_hand, bids[i]))
        elif hand_type==3:
            three_of_a_kinds.append((replaced_hand, bids[i]))
        elif hand_type==4:
            two_pairs.append((replaced_hand, bids[i]))
        elif hand_type==5:
            one_pairs.append((replaced_hand, bids[i]))
        else:
            high_cards.append((replaced_hand, bids[i]))
    
    ordered_bids = []
    for some_list in [five_of_a_kinds, four_of_a_kinds, full_houses, three_of_a_kinds, two_pairs, one_pairs, high_cards]:
        ordered_bids += order_list(some_list)
    
    total_winnings = 0
    for i in range(len(ordered_bids)):
        total_winnings += (i+1)*ordered_bids[::-1][i]

    return total_winnings

In [8]:
total_winnings()

func:'total_winnings' args:[(), {}] took: 0.0000319481 sec


6440

## Part 2

In [9]:
hands = [line.split(' ')[0] for line in lines]
bids = [int(line.split(' ')[1]) for line in lines]

In [10]:
new_ranked_cards = ['A', 'K', 'Q', 'T', '9', '8', '7', '6', '5', '4', '3', '2', 'J']
new_replacements = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]

In [11]:
def max_card(hand):
    max_card = 'J'
    for card in hand:
        card_index = new_ranked_cards.index(card)
        if card_index < new_ranked_cards.index(max_card):
            max_card = card
    return max_card

def new_revert_replace_hand(hand):
    new_hand = ''
    for card in hand:
        new_hand += new_ranked_cards[new_replacements.index(card)]
    return new_hand

def best_hand(hands):
    replaced_hands = [list(new_replace_hand(hand)) for hand in hands]
    hands_sorted=sorted(replaced_hands, key=lambda x: (x[0], x[1], x[2], x[3], x[4]))
    reverted_hand = new_revert_replace_hand(hands_sorted[0])
    return reverted_hand

In [12]:
def max_count_card(hand):
    max_count = 0
    for card in hand:
        card_count = hand.count(card)
        if card_count > max_count:
            max_count = card_count
    return max_count

def new_determine_type(hand):
    set_hand = set(hand)
    list_set_hand = list(set_hand)
    num_of_jokers = hand.count('J')

    # find out which card to replace
    if num_of_jokers > 0:
        possible_hands = []
        possible_sets = []
        for card in list_set_hand:
            hand1 = list(map(lambda x: x.replace('J', card), hand))
            hand1 = ''.join(hand1)
            set_hand1 = set(hand1)
            possible_hands.append(hand1)
            possible_sets.append(set_hand1)

        # rank the possible hands
        possible_set_lengths = [len(set) for set in possible_sets]
        min_possible_set_length = min(possible_set_lengths)
        possible_hands = [hand for hand in possible_hands if len(set(hand))==min_possible_set_length]
        max_count = max([max_count_card(hand) for hand in possible_hands])
        possible_hands = [hand for hand in possible_hands if max_count_card(hand) == max_count]
        if len(possible_hands) > 1:
            hand = best_hand(possible_hands)
            set_hand = set(hand)
            list_set_hand = list(set_hand)
        else:
            hand = possible_hands[0]
            set_hand = set(hand)
            list_set_hand = list(set_hand)
    
    if len(set_hand)==1:
        return 0
    if len(set_hand)==2:
        if hand.count(list_set_hand[0])==4 or hand.count(list_set_hand[0])==1:
            return 1
        else:
            return 2
    if len(set_hand)==3:
        if hand.count(list_set_hand[0])==3 or hand.count(list_set_hand[1])==3 or hand.count(list_set_hand[2])==3:
            return 3
        else:
            return 4
    if len(set_hand)==4:
        return 5
    if len(set_hand)==5:
        return 6

def new_replace_hand(hand):
    new_hand = []
    for card in hand:
        new_hand.append(new_replacements[new_ranked_cards.index(card)])
    return new_hand

In [13]:
@timing
def new_total_winnings():
    five_of_a_kinds = []
    four_of_a_kinds = []
    full_houses = []
    three_of_a_kinds = []
    two_pairs = []
    one_pairs = []
    high_cards = []
    
    for i, hand in enumerate(hands):
        hand_type = new_determine_type(hand)
        replaced_hand = new_replace_hand(hand)
        if hand_type == 0:
            five_of_a_kinds.append((replaced_hand, bids[i]))
        elif hand_type == 1:
            four_of_a_kinds.append((replaced_hand, bids[i]))
        elif hand_type==2:
            full_houses.append((replaced_hand, bids[i]))
        elif hand_type==3:
            three_of_a_kinds.append((replaced_hand, bids[i]))
        elif hand_type==4:
            two_pairs.append((replaced_hand, bids[i]))
        elif hand_type==5:
            one_pairs.append((replaced_hand, bids[i]))
        else:
            high_cards.append((replaced_hand, bids[i]))
    
    ordered_bids = []
    for some_list in [five_of_a_kinds, four_of_a_kinds, full_houses, three_of_a_kinds, two_pairs, one_pairs, high_cards]:
        ordered_bids += order_list(some_list)
    
    total_winnings = 0
    for i in range(len(ordered_bids)):
        total_winnings += (i+1)*ordered_bids[::-1][i]
        
    return total_winnings

In [14]:
new_total_winnings()

func:'new_total_winnings' args:[(), {}] took: 0.0000686646 sec


5905

## Michael's Way

In [15]:
from pathlib import Path
from collections import Counter

In [16]:
data = [line.split() for line in lines]

In [17]:
values = {'T': 10,
          'J': 11,
          'Q': 12,
          'K': 13,
          'A': 14}

def hand_rank(hand):
    count = sorted(Counter(hand).values(), reverse = True)
    
    hand = [int(card) if card.isdigit() else values[card] for card in hand]
    
    return count, hand

In [18]:
ts = time()
ranked_hands = sorted(data, key = lambda x: hand_rank(x[0]))
print(sum([(i+1)*int(bid) for i, (hand, bid) in enumerate(ranked_hands)]))
te = time()
print('took: %2.10f sec' % (te-ts))

6440
took: 0.0005183220 sec


In [19]:
values = {'T': 10,
          'J': 0,
          'Q': 12,
          'K': 13,
          'A': 14}

def hand_rank2(hand):

    jokers = hand.count('J')
    count = sorted(Counter(hand.replace('J', '')).values(), reverse = True)

    if not count: # All Joker hand
        count = [0]
        
    count[0] += jokers
    
    hand = [int(card) if card.isdigit() else values[card] for card in hand]

    return count, hand


In [20]:
ts = time()
ranked_hands = sorted(data, key = lambda x: hand_rank2(x[0]))

print(sum([(i+1)*int(bid) for i, (hand, bid) in enumerate(ranked_hands)]))
te = time()
print('took: %2.10f sec' % (te-ts))

5905
took: 0.0003509521 sec
